In [ ]:
# data downloading
import datetime as dt
from remodels.data.EntsoeApi import EntsoeApi

start_date = dt.date(2015, 1, 1)
end_date = dt.date(2017, 1, 1)
security_token = "7032e795-c8ae-4a50-aac8-a377b64b1c9e"  #"..."  # free token from https://transparency.entsoe.eu/
entsoe_api = EntsoeApi(security_token)
prices = entsoe_api.get_day_ahead_pricing(
    start_date,
    end_date,
    "10Y1001A1001A63L",  # Germany domain code in ENTSO-E
    resolution_preference=60,  # resolution in minutes
)
forecast_load = entsoe_api.get_forecast_load(
    start_date, 
    end_date, 
    "10Y1001A1001A63L"
)
germany_data = prices.join(forecast_load)

In [ ]:
# VST & point predictions
from remodels.pipelines.RePipeline import RePipeline
from remodels.pointsModels import PointModel
from remodels.transformers import StandardizingScaler
from remodels.transformers.VSTransformers import ArcsinhScaler
from sklearn.linear_model import LinearRegression

pipeline = RePipeline(
    [
        ("standardScaler", StandardizingScaler()),
        ("vstScaler", ArcsinhScaler()),
        ("linearRegression", LinearRegression()),
    ]
)
pointModel = PointModel(pipeline=pipeline)
pointModel.fit(germany_data, dt.date(2016, 12, 1), dt.date(2016, 12, 31))
point_predictions = pointModel.predict(calibration_window=182)

In [ ]:
# point predictions metrics
pointModel.summary()

In [ ]:
# probabilistic predictions
from remodels.qra import QRA
from remodels.qra.tester import QR_Tester

# actual prices - target variable in QRA model
price_da = point_predictions.join(germany_data)["price_da"]
# selected QRA variant
qra_model = QRA(fit_intercept=True)

results = QR_Tester(
    calibration_window=72,
    qr_model=qra_model
).fit_predict(point_predictions, price_da)

In [ ]:
# probabilistic predictions metrics
# alpha=50 is excpected coverage of prediction intervals
results.aec(alpha=50)  # Average Empirical Coverage

In [ ]:
results.kupiec_test(alpha=50, significance_level=0.05) 

In [ ]:
results.christoffersen_test(alpha=50, significance_level=0.05)

In [ ]:
results.aps()  # Aggregate Pinball Score